In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

In [ ]:
# SEED for reproducibility
SEED = 1
torch.manual_seed(SEED)
cuda = torch.cuda.is_available()
if cuda:
    torch.cuda.manual_seed(SEED)

# Transformations
train_transforms = transforms.Compose([
    transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Datasets
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)
# Data Loaders
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

# Network
dropout_value = 0.1

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, bias=False), nn.ReLU(),
            nn.BatchNorm2d(16), nn.Dropout(dropout_value)
        )  # 26x26
        
        self.convblock2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, bias=False), nn.ReLU(),
            nn.BatchNorm2d(32), nn.Dropout(dropout_value)
        )  # 24x24
        
        self.convblock3 = nn.Sequential(
            nn.Conv2d(32, 10, 1, bias=False)
        )  # Transition layer: 24x24 -> 24x24
        
        self.pool1 = nn.MaxPool2d(2, 2)  # 12x12
        
        self.convblock4 = nn.Sequential(
            nn.Conv2d(10, 16, 3, bias=False), nn.ReLU(),
            nn.BatchNorm2d(16), nn.Dropout(dropout_value)
        )  # 10x10
        
        self.convblock5 = nn.Sequential(
            nn.Conv2d(16, 16, 3, bias=False), nn.ReLU(),
            nn.BatchNorm2d(16), nn.Dropout(dropout_value)
        )  # 8x8
        
        self.convblock6 = nn.Sequential(
            nn.Conv2d(16, 16, 3, bias=False), nn.ReLU(),
            nn.BatchNorm2d(16), nn.Dropout(dropout_value)
        )  # 6x6
        
        self.gap = nn.AvgPool2d(6)  # 1x1
        
        self.fc = nn.Sequential(
            nn.Conv2d(16, 10, 1, bias=False)
        )

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.gap(x)
        x = self.fc(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)


In [ ]:
# Initialize model, optimizer, and scheduler
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

# Training and Testing Functions
train_losses, test_losses = [], []
train_acc, test_acc = [], []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        train_losses.append(loss.item())
        train_acc.append(100 * correct / processed)
        pbar.set_description(desc=f'Epoch {epoch} Loss={loss.item():.4f} Accuracy={100*correct/processed:.2f}%')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    test_acc.append(100. * correct / len(test_loader.dataset))
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.2f}%)\n')


In [ ]:
# Training Loop
EPOCHS = 20
target_accuracy = 99.4  # Target accuracy percentage

for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)
    
    # Check if test accuracy exceeds the target accuracy
    if test_acc[-1] > target_accuracy:
        print(f"Target accuracy of {target_accuracy}% achieved. Stopping training.")
        break
